In [2]:
import requests
import time
from bs4 import BeautifulSoup 

In [6]:
_URL = 'https://lotv.spawningtool.com/replays/?p=1&query=&after_time=&before_time=&after_played_on=&before_played_on=&coop=n&patch=&order_by='


In [31]:
resp = requests.get(_URL)

In [37]:
soup = BeautifulSoup(resp.text, "html.parser")
dl_link = soup.findAll(class_='download-replays')
dl_link[0

[<a class="download-replays" href="/zip/?coop=&amp;after_played_on=&amp;after_played_on=&amp;patch=&amp;patch=&amp;after_time=&amp;after_time=&amp;query=&amp;query=&amp;before_time=&amp;before_time=&amp;before_played_on=&amp;before_played_on=&amp;order_by=%2Fzip%2F%3Fcoop%3D&amp;order_by=&amp;p=1" rel="nofollow" title="Download these replays"><i class="glyphicon glyphicon-download-alt"></i></a>]

https://lotv.spawningtool.com/zip/?coop=&after_played_on=&after_played_on=&patch=&patch=&after_time=&after_time=&query=&query=&before_time=&before_time=&before_played_on=&before_played_on=&order_by=%2Fzip%2F%3Fcoop%3D&order_by=&p=1
https://lotv.spawningtool.com/zip/?coop=&after_played_on=&after_played_on=&patch=&patch=&after_time=&after_time=&query=&query=&before_time=&before_time=&before_played_on=&before_played_on=&order_by=%2Fzip%2F%3Fcoop%3D&order_by=&p=2

In [33]:

dl_url = _URL + dl_link[0]['href']
dl_url

'https://lotv.spawningtool.com/replays/?p=&query=&after_time=&before_time=&after_played_on=&before_played_on=&coop=&patch=&order_by=/zip/?coop=&after_played_on=&patch=&after_time=&query=&before_time=&before_played_on=&order_by=&p=1'

In [7]:
_PATH = '../get_replay_data/data/SpawningTool/Other'
dl_url = 'https://lotv.spawningtool.com/zip/?after_played_on=01%2F01%2F2019&before_time=&patch=&after_time=&before_played_on=&coop=n&query=&order_by=&p='

for i in range(1, 1011):
    resp = requests.get(dl_url + str(i))
    with open(_PATH + f'/page{i}.zip', 'wb') as f:
        f.write(resp.content)
    
    display(i)

1

2

3

4

5

...

1006

1007

1008

1009

1010

In [ ]:
# Extract ZIP files and build directory of files and folders